In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [126]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
import edhec_risk_kit as erk

In [128]:
# using seaborn style (type plt.style.available to see available styles)
plt.style.use("seaborn-dark")

## Time value of money - Present and future value

We recall from week 1, that is, in general, for given a time frame $(t, t+k)$, with $k>1$, the **compound return** over the time frame is given by:
$$
R_{t,t+k} = (1+R_{t,t+1})(1+R_{t+1,t+2})\cdots(1+R_{t+k-1,t+k}) - 1, 
$$
and if single returns are all the same over time periods, say $R$, then the formula for compound returns simply becomes: 
$$
R_{t,t+k} = (1+R)^{k} - 1.
$$

Therefore, if we suppose that $P_t$ was the price a t time $t$ and $P_{t+k}$ is the time when all returns have been compounded, we know that:
$$
P_{t+k} = P_t(1 + R_{t, t+k}) = 
\begin{cases}
& P_t \; (1+R_{t,t+1})(1+R_{t+1,t+2})\cdots(1+R_{t+k-1,t+k})\;  & \text{if returns are different}, \\
& P_t \; (1+R)^{k} \;  & \text{if all returns are equal to $R$}.
\end{cases}
$$

In general, we say that $P_{t+k}$ is the **future value (FV)** of the amount of money equal to $P_t$. Likewise, we say that 
$P_t$ is the **present value (PV)** of the amount of money $P_{t+k}$ due in the future (at time $t+k$).

For example, $100\$$ invested for one year and earning $5\%$ interest will be worth $105\$$ after one year. 
Therefore, **both $100\$$ paid now and $105\$$ paid exactly one year later have the same value to a recipient who expects $5\%$ 
interest** (assuming that inflation would be zero): that is, $100\$$ (PV) invested for one year at $5\%$ 
interest has a future value (FV) of $105\$$.

Let us consider the case of equal returns $R$ over time. From the equation above we see that to obtain present value PV (i.e., 
the amount that we would have to invest today) we have to **discount** the future value FV (or payment amount) by the interest rate for the period. Hence:
$$
FV = PV(1 + R)^k
\qquad\text{and}\qquad
PV = \frac{FV}{(1 + R)^k}.
$$

The factor
$$
\frac{1}{(1+R)^k}
$$
is called the **discount rate**.

**EXAMPLE**: suppose we are giving an option to receive $1000\$$ today and receiving the same amount of $1000\$$ in three years. 
*What we would choose?*

**We should get the money today, of course**. In fact, receiving $1000\$$ today means that the present value is $PV=1000$, and this amount could be invested and become bigger in three years. Suppose we invest it for three year at annual rate of $3\%$. 
The future value of $1000\%$ today will be 
$$
FV = 1000 (1 + 0.03)^3 = 1.092.7\$. 
$$
If we instead decide to get the money in three years, basically the amount $1000\%$ is already our future value. That is, supposing the same interest rate, the present value of $1000\$$ in three years is:
$$
PV = \frac{1000}{(1+0.03)^3} = 915,14\$, 
$$
and this means that today, we would accept less money ($915,14\$ < 1000\$$).


**EXAMPLE**: suppose we are giving an option to receive $1000\$$ today or receiving $1100\$$ in three years. 
*What we would choose now?*

This is more difficult now, because we get $1000\$$ we may end up with an amount of money which is less than $1152$ in three years. 
In fact, if we suppose that our hypotetical investment still guarantee an annual $3\%$ interest rate, it means that the present value of 
$1150\$$ after three years is:
$$
PV = \frac{1150}{(1+0.03)^3} = 1052.41\$.
$$
That is, taking $1150\$$ in three years is equivalent get $1052.41\$$ today, which is an amount larger than $1000\$$. 
**We will then decide to get the money later**.

### Cumulative present value (of future cash flows)

The **cumulative present value** (PV) of future cash flows can be calculated by summing the contributions of future values $FV_t$, i.e., the value of cash flow at time $t$. 

Suppose that company A owes to company B a total of $25000\$$ over three years divided in this way: $8000\$$ in the first year, $11000\$$ in the second year, and $6000\$$ in third year. Suppose that company B applies a $5\%$ annual interest rate. 
What is the present value of such **liability**?. 

We know that the present value of an amount of $FV_1 := 8000\$$ in one year is given by $PV_1 = 8000 / (1+0.05)=7619.05\$$. 
Analogously, the present value of an amount of $FV_2 := 11000\$$ in two years is by $PV_2 = 11000 / (1+0.05)^2=9977.33\$$. 
And finally, the present value of an amount of $FV_3 := 6000\$$ in three years is by $PV_3 = 6000 / (1+0.05)^3=5183.03\$$. 

Wrapping up, the present value of the entire liability is given by the sum of single present values of future cashflow, i.e., 
about $PV_1+PV_2+PV_3 = 22779.4\$$.  

In general, if we **denote the future cash flows as the future values**, we have that the present value of 
a **liability** $L$ is:
$$
PV(L) = \sum_{i=1}^N \frac{FV_{t_i}}{(1+R)^{t_i}} := \sum_{i=1}^N B(t_i) L_{t_i}, 
$$
and $L_{t_i}$ denotes the future cash flow at time $t_i$, i.e., the **liability at time $t_i$**, and 
$B(t_i) := 1 / (1+R)^{t_i}$ denotes the **discount rate**.

In [5]:
# Suppose we want to get the PV of 1 dollar paid in 10years with an annual rate of r = 3%
PV = erk.discount(10, 0.03)
PV

0.7440939148967249

In [6]:
# that is, PV = 0.74 cents means that in 10 years will be FV = 1 (considering r = 3%). In fact:
FV = PV * (1+0.03)**10
FV

1.0

In [7]:
def present_value(L, r):
    '''
    Computes the PV of a sequence of liabilities where L is the sequence of 
    liabilities 
    '''
    dates = L.index
    discounts = erk.discount(L.index, r) # this is the series of present values of future cashflows
    return np.dot(discounts, L )         # = (discounts * L).sum()

In [8]:
L = pd.DataFrame([8000,11000,6000], index=[1,2,3], columns=["CFs"])
L.index.name = "years"
L

,CFs
years,
1,8000
2,11000
3,6000


In [9]:
# the total value of the liability is:
L.sum()

CFs    25000
dtype: int64

In [10]:
# whereas the present value is:
PV = present_value(L, 0.05)
PV

array([22779.39747328])

that is, I owe $25000\$$ in the future (in three years), that's the total value of the liability, but its present value 
is about $22779.4\$$. 

This is means that if I had $22779.4\$$ today, I have no problem in paying off my liability since I know what interest rates are. 
I would have a problem in case today I had less than $22779.4\$$ because with the current interest rates I will not be able to pay $25000$ in three years.

### Funding ratio

The funding ratio is therefore a simply ratio between the value of assets I currently hold and the present value of my liability.
$$
FR = \frac{\text{assets}}{{PV(L)}}.
$$
If the value of my current assets if bigger than $PV(L)$, then $FR >1$ and this means that I have enough funds to pay of my liability in the coming future. In the example above:

In [11]:
def funding_ratio(asset_value, liabilities, r):
    '''
    Computes the funding ratio between the value of holding assets and the present 
    value of the liabilities given an interest rate r
    '''
    return asset_value / present_value(liabilities, r)

In [12]:
asset = 20000 
funding_ratio(asset, L, 0.05)

array([0.87798635])

that is, if I currently have $20000\$$ I won't be able to payoff $25000\$$ in three years. On the other hand:

In [13]:
asset = 23500 
funding_ratio(asset, L, 0.05)

array([1.03163396])

in this case we have enough funds to fullfil our liability since its present value is $22779.4\$$ and the value of our current asset
is $23500\$$.

In [14]:
def show_funding_ratio(asset, r):
    fr = funding_ratio(asset, L, r)
    
    print("Funding ratio: {:.3f}".format(float(fr)))
    
    fig, ax = plt.subplots(1,2,figsize=(15,4))
    ax[0].scatter(r, fr)
    ax[0].set_xlabel("rates")
    ax[0].set_ylabel("funding ratio")
    ax[0].set_xlim([0.0, 0.06])
    ax[0].set_ylim([0.70, 1.2])
    ax[0].plot([r,r],[0,fr], color="b", alpha=0.5)
    ax[0].plot([0,r],[fr,fr], color="b", alpha=0.5)
    ax[0].grid()
    
    ax[1].scatter(asset, fr)
    ax[1].set_xlabel("assets")
    ax[1].set_ylabel("funding ratio")
    ax[1].set_xlim([19000, 27000])
    ax[1].set_ylim([0.70, 1.2])
    ax[1].plot([asset,asset],[0,fr], color="b", alpha=0.5)
    ax[1].plot([0,asset],[fr,fr], color="b", alpha=0.5)
    ax[1].grid()
    
fr_controls = widgets.interact(show_funding_ratio, 
                               asset = widgets.FloatSlider(min=19000, max=27000, step=250, value=21000),
                               r = (0.0, 0.06, 0.005)
                              )

interactive(children=(FloatSlider(value=21000.0, description='asset', max=27000.0, min=19000.0, step=250.0), F…

### Nominal rate and Effective annual interest rate

Before introducing a stochastic equation modelling the change in interest rates, let us for a second come back to compound returns in order to look at them from another point of view which changes nomenclature. 

#### Short-rate v.s. Long-Rate (annualized)
Recall that given a fixed return $R$, the compound return after a period of length $t+k$ (starting from $t$) is given by:
$$
R_{t,t+k} = (1+R)^{k} - 1.
$$

For example, suppose that we borrow some money, say $P := 1\$$, during one year at annual interest rate of $r_{ann} := 10\%$. 
Also, suppose that we have to pay back in one solution at the end of the year. This means that the return after one year is $10\%$ and
we have to pay back:
$$
1 + 0.1\cdot 1 = 1.1\$.
$$
Now, suppose that we still borrow $P$ at interest rate $r := 10\%$ but with have to pay in two solutions, i.e., every $6$ months. 
Hence, we get the semi-annual rate $r/2 = 5\%$ which gives a total compound return of 
$$
\left(1 + \frac{r}{2}\right)^2 -1 = 0.1025\$,
$$
and then we have to pay back $1 + 0.1025\cdot 1= 1.1025\$$.
If, in the same case, payments were due every month, we would have:
$$
\left(1 + \frac{r}{12}\right)^{12}-1 = 0.1047\$,
$$
and then we have to pay back $1 + 0.1047\cdot 1= 1.1047\$$.

We can see that due to compounding, **the more payments due over the entire period the larger the total compound return, hence more money have to be paid back**. 

In general, given a **nominal interest rate** $r$ (also called **instantaneous interest rate**) and $N$ **number of periods**, i.e., the number of payments (of the investment, loan, and so on), 
the total return is given by:
$$
R = \left(1 + \frac{r}{N}\right)^N - 1.
$$
Such $R$ is nothing but that the **annualized return** (as we used to call it so far) or the **effective annual interest rate**.

In [114]:
# Consider a nominal interest rate of 10% and monthly payments
nominal_rate = 0.1
periods_per_year = 12

# that is, every month, we have the following rates
rets = pd.DataFrame( [nominal_rate/periods_per_year for i in range(10)] )
rets.head(3)

,0
0,0.008333
1,0.008333
2,0.008333


Due to compounding, we have:

In [115]:
ann_ret = erk.annualize_rets(rets, periods_per_year)
ann_ret

0    0.104713
dtype: float64

In [116]:
R = (1 + nominal_rate / periods_per_year)**periods_per_year - 1
np.round(R,6)

0.104713

Notice that we got, obviously, the same number (and that it does not depend on the number of returns in the dataframe...). 

By having the annualized return expressed in the form above, we realize that when $N$ becomes large we have:
$$
\lim_{N\to \infty} \left(1 + \frac{r}{N} \right)^N = e^{r}, 
$$
and then 
$$
1 + R \approx e^r.
$$
Then we have:
$$
R \approx e^r - 1
\quad\text{and}\quad 
r \approx \log{(1 + R)}.
$$

## CIR model: simulate changes in interest rates

The **CIR model** (from **Cox, Ingersoll, Ross**) is used to simulate changes in interest rates (returns) and it is an extension of the **Vasicek** model to prevent negative interest rates. It is a type of **one factor model**, or **short-rate model**, 
as it describes interest rate movements as driven by only one source of market risk. 
The model can be used in the valuation of interest rate derivatives. 

The dynamic for interest rates is modelled in the following way:
$$
dr_t = a(b-r_t)dt + \sigma\sqrt{r_t}dW_t,
$$
where, $W_t$ is a Brownian motion (see Week 3) which models the random market risk factor, $b$ is a **(long-term) mean interest rate**, and the difference $b-r_t$ denotes how far away is the current interest rate from the (long-term) mean interest rates, ensuring mean reversion of the interest rate towards $b$.
Finally, $a$ is the **mean-reversion speed**, or speed of adjustment to the mean, which models **how fast** do we revert to the (long-term) mean interest rate.

Hence, movements of interest rates depend on long-term mean rates (by $b$) and how fast the dynamic changes and try to get close to the mean rate (by $a$) after deviations (by the noise given by $\sigma\sqrt{r_t}dW_t$). 
Note that the standard deviation factor $\sigma\sqrt{r_t}$ avoids the possibility of negative interest rates for all positive values of $a$ and $b$. Moreover, an interest rate equal to zero is also precluded if $2ab\geq \sigma^2$. 

More generally, when the rate $r_t$ is close to zero, the standard deviation $\sigma\sqrt{r_t}$ also becomes very small, which dampens the effect of the random shock on the rate. Consequently, in this case, the evolution of the rate becomes dominated by the drift factor 
which pushes the rate upwards (towards equilibrium).

In [246]:
def simulate_cir(n_years=10, n_scenarios=10, a=0.5, b=0.03, sigma=0.05, periods_per_year=12, r0=None):
    '''
    Evolution of an initial stock price using Geometric Brownian Model:
        (S_{t+dt} - S_t)/S_t = mu*dt + sigma*sqrt(dt)*xi,
    where xi are normal random variable N(0,1). 
    The equation for percentage returns above is used to generate returns and they are compounded 
    in order to get the prices.    
    Note that default periods_per_year=12 means that the method generates monthly prices (and returns):
    change to 52 or 252 for weekly or daily prices and returns, respectively.
    The method returns a dataframe of prices and the dataframe of returns.
    '''
    
    if r0 is None:
        r0 = b
        
    # Compute the price of a ZCB
    def zcbprice(ttm,r,h):
        A = ( ( 2*h*np.exp(0.5*(a+h)*ttm) ) / ( 2*h + (a+h)*(np.exp(h*ttm)-1) ) )**(2*a*b/(sigma**2))
        B = ( 2*(np.exp(h*ttm)-1) ) / ( 2*h + (a+h)*(np.exp(h*ttm)-1) ) 
        return A * np.exp(-B * r)
    
    dt = 1 / periods_per_year
    n_steps = int(n_years * periods_per_year) + 1
    
    # get the nominal (instantaneous) rate 
    r0 = erk.annual2nomimal_rate_gen(r0)
    
    # the schock is sqrt(dt)*xi_t, with xi_t being standard normal r.v.
    shock = np.random.normal(loc=0, scale=(dt)**(0.5), size=(n_steps, n_scenarios))
    
    # Rates initialization
    rates = np.zeros_like(shock)
    rates[0] = r0 
    
    # Price initialization and parameters
    zcb_prices = np.zeros_like(shock)
    h = np.sqrt(a**2 + 2*sigma**2)
    zcb_prices[0] = zcbprice(n_years,r0,h)


    for step in range(1,n_steps):
        # previous interest rate
        r_t = rates[step-1]
        
        # Current (updated) interest rate: CIR equation
        rates[step] = r_t + a*(b - r_t) + sigma*np.sqrt(r_t)*shock[step]
        
        # Current (updated) ZCB price
        zcb_prices[step] = zcbprice(n_years - dt*step, r_t, h)       
 
    rates = pd.DataFrame( erk.nominal2annual_rate_gen(rates) )
    zcb_prices = pd.DataFrame( zcb_prices )
    return rates, zcb_prices


def show_cir(n_years=10, n_scenarios=10, a=0.05, b=0.05, sigma=0.04, periods_per_year=12, r0=None):
    '''
    '''
    rates, zcb_price = simulate_cir(n_years=n_years, n_scenarios=n_scenarios, a=a, b=b, sigma=sigma, periods_per_year=periods_per_year, r0=r0)
    
    fig, ax = plt.subplots(1,2,figsize=(20,5))
    rates.plot(ax=ax[0], grid=True, title="CIR model: interest rates", color="indianred", legend=False)
    zcb_price.plot(ax=ax[1], grid=True, title="CIR model: ZCB price", color="indianred", legend=False)


In [247]:
cir_controls = widgets.interact(show_cir, 
                                n_years = (1, 10, 1), 
                                n_scenarios = (1, 200, 1), 
                                a = (0.005, 1, 0.005), 
                                b = (0.002, 0.15, 0.001), 
                                sigma = (0.001, 0.15, 0.001), 
                                periods_per_year = [12, 52, 252], 
                                r0 = (0.002, 0.30, 0.01)
                               )

interactive(children=(IntSlider(value=10, description='n_years', max=10, min=1), IntSlider(value=10, descripti…

### Using the CIR model for zero-coupon bond pricing

Under the **no-arbitrage assumption**, a **zero-coupon bond** may be priced using the interest rate process modeled by the CIR model. 
The bond price $P(t,T)$ of a zero-coupon bond with maturity $T$ is exponential affine in the interest rate and is given by:
$$
P(t,T) = A(t,T)e^{-B(t,T)r_t}, 
$$
where
$$
\begin{align}
A(t,T) &:= \left( \frac{ 2h e^{(a+h)\tau/2}  }{ 2h+(a+h)(e^{h\tau}-1) }   \right)^{2ab/\sigma^2}, \\
B(t,T) &:= \frac{ 2(e^{h\tau} - 1)  }{ 2h+(a+h)(e^{h\tau}-1) },  \\
h &:= \sqrt{a^2 + 2\sigma^2}, \\
\tau &:= T - \tau.
\end{align}
$$
I hope to come back to this sooner or later.

In [119]:
r = 0.1
periods_per_year = 12

In [120]:
nominal2annual_rate(r, periods_per_year)

0.10471306744129683

In [121]:
nominal2annual_rate_gen(r)

0.10517091807564771

In [123]:
R = 0.10471306744129683
periods_per_year = 12

In [124]:
annual2nomimal_rate(R, periods_per_year)

0.09999999999999964

In [125]:
annual2nomimal_rate_gen(R)

0.09958563377634075

## Liability hedging 